In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

In [4]:
import logging

# Set the logging level to ERROR
logging.getLogger("py4j").setLevel(logging.ERROR)

In [5]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Preprocess") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "75g") \
    .config("spark.pyspark.python", "/usr/local/bin/python3.9") \
    .config("spark.pyspark.driver.python", "/usr/local/bin/python3.9") \
    .getOrCreate()

# Read CSV into DataFrame
# prices = spark.read.csv("itineraries.csv", header=True, inferSchema=True)

In [6]:
flight_prices = spark.read.csv("itineraries.csv", header=True, inferSchema=True)

In [19]:
flight_prices_3 = flight_prices

In [123]:

def duration_to_minutes_2(duration_str):
    try:
        parts = duration_str.split('T')
        parts_l = parts[0].split('P')
        days = int(parts_l[1][0])
    except:
        days = 0

    try:
        parts = duration_str.split('T')
        parts_r = parts[1].split('H')
        hours = int(parts_r[0]) if len(parts_r) > 1 else 0
    except:
        hours = 0

    try:
        parts = duration_str.split('T')
        parts_r = parts[1].split('H')
        minutes = int(parts_r[-1][:-1])
    except:
        minutes = 0

    try:
        return ((24*days) + hours) * 60 + minutes
    except:
        return "error"
    # try:
    #     parts = duration_str.split('T')
    #     parts_l = parts[0].split('P')
    #     days = int(parts_l[1][0])
    #     parts_r = parts[1].split('H')
    #     hours = int(parts_r[0]) if len(parts_r) > 1 else 0
    #     minutes = int(parts_r[-1][:-1])
    #     return ((24*days) + hours) * 60 + minutes
            
            

In [131]:
print(duration_to_minutes_2("PT5H"))

300


In [132]:
# Define a UDF (User Defined Function) to convert duration strings to minutes
def duration_to_minutes(duration_str):
    try:
        parts = duration_str.split('T')
        parts_l = parts[0].split('P')
        days = int(parts_l[1][0])
    except:
        days = 0

    try:
        parts = duration_str.split('T')
        parts_r = parts[1].split('H')
        hours = int(parts_r[0]) if len(parts_r) > 1 else 0
    except:
        hours = 0

    try:
        parts = duration_str.split('T')
        parts_r = parts[1].split('H')
        minutes = int(parts_r[-1][:-1])
    except:
        minutes = 0

    try:
        return ((24*days) + hours) * 60 + minutes
    except:
        return 0                 

In [133]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

duration_to_minutes_udf = udf(duration_to_minutes, IntegerType())

In [134]:
flight_prices_2 = flight_prices.withColumn(
    "travelDurationMinutes",
    duration_to_minutes_udf("travelDuration")
)

In [135]:
from pyspark.sql.functions import col
a = flight_prices_2.filter(col("travelDurationMinutes")==0)

In [136]:
a.count()

0

In [137]:
a.limit(10).toPandas().head(10)

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,travelDurationMinutes


In [ ]:
# flight_prices.limit(1).toPandas().head(1)

In [26]:
flight_prices_2.na.drop().show()

+--------------------+----------+----------+---------------+------------------+-------------+--------------+-----------+--------------+------------+---------+--------+---------+--------------+-------------------+---------------------------------+------------------------+-------------------------------+----------------------+--------------------------+----------------------------+--------------------+-------------------+----------------------------+-------------------------+----------------+-----------------+---------------------+
|               legId|searchDate|flightDate|startingAirport|destinationAirport|fareBasisCode|travelDuration|elapsedDays|isBasicEconomy|isRefundable|isNonStop|baseFare|totalFare|seatsRemaining|totalTravelDistance|segmentsDepartureTimeEpochSeconds|segmentsDepartureTimeRaw|segmentsArrivalTimeEpochSeconds|segmentsArrivalTimeRaw|segmentsArrivalAirportCode|segmentsDepartureAirportCode| segmentsAirlineName|segmentsAirlineCode|segmentsEquipmentDescription|segmentsDura

In [9]:
flight_prices.columns

['legId',
 'searchDate',
 'flightDate',
 'startingAirport',
 'destinationAirport',
 'fareBasisCode',
 'travelDuration',
 'elapsedDays',
 'isBasicEconomy',
 'isRefundable',
 'isNonStop',
 'baseFare',
 'totalFare',
 'seatsRemaining',
 'totalTravelDistance',
 'segmentsDepartureTimeEpochSeconds',
 'segmentsDepartureTimeRaw',
 'segmentsArrivalTimeEpochSeconds',
 'segmentsArrivalTimeRaw',
 'segmentsArrivalAirportCode',
 'segmentsDepartureAirportCode',
 'segmentsAirlineName',
 'segmentsAirlineCode',
 'segmentsEquipmentDescription',
 'segmentsDurationInSeconds',
 'segmentsDistance',
 'segmentsCabinCode']

In [138]:
ohe_cols = ["startingAirport","destinationAirport"]

In [139]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline


indexers = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid="keep") for column in ohe_cols]

# Perform OneHotEncoding on the indexed columns
encoders = [OneHotEncoder(inputCol=column+"_index", outputCol=column+"_onehot") for column in ohe_cols]

# Create a pipeline to execute the indexers and encoders sequentially
pipeline = Pipeline(stages=indexers + encoders)

# Fit the pipeline to the data and transform the DataFrame
model = pipeline.fit(flight_prices)
encoded_df = model.transform(flight_prices)

# Show the encoded DataFrame
encoded_df.show()


ERROR:root:KeyboardInterrupt while sending command.=>           (184 + 8) / 232]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [140]:
# Assuming 'spark' is your SparkSession object
spark.stop()


24/05/01 00:22:15 WARN DiskBlockObjectWriter: Error deleting /private/var/folders/gd/dnbt5zx96z3458pcdndk061c0000gn/T/blockmgr-4178c8a9-c145-4d09-a76f-41facc9b2cb2/12/temp_shuffle_f8243f8e-d7c4-4efc-b499-6967cc8f1b6d
24/05/01 00:22:15 ERROR Utils: Uncaught exception in thread Executor task launch worker for task 192.0 in stage 50.0 (TID 3231)
java.lang.NullPointerException: Cannot invoke "org.apache.spark.SparkEnv.blockManager()" because the return value of "org.apache.spark.SparkEnv$.get()" is null
	at org.apache.spark.scheduler.Task.$anonfun$run$3(Task.scala:146)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.Task.run(Task.scala:144)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)
	at org.apache.spark.util.Utils$.tryWithS